In [45]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import Callbacks

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model


model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Only use the 100 batches per epoch (that's 64 * 100 samples)
model.fit(train_dataset, epochs=20, callbacks=[Callbacks.tboard_callback])

Epoch 1/20
  7/782 [..............................] - ETA: 15s - loss: 2.1828 - sparse_categorical_accuracy: 0.2583

2022-02-10 11:22:04.687320: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-02-10 11:22:04.687471: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-02-10 11:22:04.845391: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-02-10 11:22:04.855988: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-02-10 11:22:04.866670: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20220210-112142/train/plugins/profile/2022_02_10_11_22_04
2022-02-10 11:22:04.872642: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20220210-112142/train/plugins/profile/2022_02_10_11_22_04/zeissstudent1.trace.json.gz
2022-02-10 11:22:04.876620: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20220210-112142/train/plugins/profile/20

782/782 [==============================] - 2s 3ms/step - loss: 0.5788 - sparse_categorical_accuracy: 0.8405
Epoch 2/20
782/782 [==============================] - 2s 3ms/step - loss: 0.1786 - sparse_categorical_accuracy: 0.9485
Epoch 3/20
782/782 [==============================] - 3s 4ms/step - loss: 0.1265 - sparse_categorical_accuracy: 0.9633
Epoch 4/20
782/782 [==============================] - 2s 2ms/step - loss: 0.1008 - sparse_categorical_accuracy: 0.9696
Epoch 5/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0834 - sparse_categorical_accuracy: 0.9759
Epoch 6/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0699 - sparse_categorical_accuracy: 0.9796
Epoch 7/20
782/782 [==============================] - 3s 4ms/step - loss: 0.0608 - sparse_categorical_accuracy: 0.9814
Epoch 8/20
782/782 [==============================] - 2s 3ms/step - loss: 0.0533 - sparse_categorical_accuracy: 0.9837
Epoch 9/20
782/782 [==============================] - 3s 4m

In [58]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Only take one and zeros
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [60]:
import models
model = models.classiRaw3D(training_dataset.element_spec[0].shape, None, reconstruction=False)

(55000,)

In [23]:
total_size = 1536 * 2047 * 2044 * 2
print("Total Size: " + str(total_size))

last_element_begin = (88 * 23 + 2 * 72 * 28 * 2047) * 1536
print("Last Element Begin: " + str(last_element_begin))

last_bscan_last_element = 2 * 1536 * 2047 * 27 + last_element_begin + 1536 * 23
print("Differenz: " + str(total_size - last_bscan_last_element))
print("Should be: 1 bscan vor Ende: 3144192")

Total Size: 12853456896
Last Element Begin: 12680491008
Differenz: 3144192
Should be: 1 bscan vor Ende: 3144192


In [62]:
int(["1", "2"])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [43]:
import InputList
import BinaryReader
import time
import tensorflow as tf

def benchmark(benchmark_dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in benchmark_dataset:
            pass
            # Performing a training step
            #time.sleep(0.01)

    print("Execution time:", time.perf_counter() - start_time)

reader = BinaryReader.BinaryReader()
ds = reader.create_test_dataset(InputList.diabetic_training_files[0:2]).batch(10).take(50)

# THIS IS ONLY GOOD WITH GPU
optimized_ds = tf.data.Dataset.range(2)\
    .interleave(lambda _: ds,
                num_parallel_calls=tf.data.AUTOTUNE)\
    .prefetch(
        tf.data.AUTOTUNE
    )

benchmark(ds)
benchmark(optimized_ds)

Execution time: 6.725447240984067
Execution time: 19.533383095054887


In [15]:
import numpy as np
a = np.array([[1,2,0], [1,0,0], [0,1,0]])

In [23]:
b = ((a == 0) | (a == 1)).all(axis=0)
a[:, b]

array([[1, 0],
       [1, 0],
       [0, 0]])

In [25]:
b

array([ True, False,  True])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(elem[0][0,:].numpy().reshape(28, 28), cmap="Greys")

In [ ]:
def print_bag():
    model.evaluate(x_test, y_test)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(x_train, y_train, epochs=3, callbacks=[tensorboard_callback])

In [ ]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):

    def on_train_begin(self, logs=None):
        plt.figure()
        self.loss_history = []

    def on_train_batch_end(self, batch, logs=None):
    #     print(
    #         "Up to batch {}, the average loss is {:7.2f}.".format(batch, logs["loss"])
    #     )
        self.loss_history.append(logs["loss"])
        plt.plot(self.loss_history)

    def on_test_batch_end(self, batch, logs=None):
        print(
            "Up to batch {}, the average loss is {:7.2f}.".format(batch, logs["loss"])
        )

In [26]:
import numpy as np
a = [1, 2, 3, 3, 3]
np.argmax(a)

2

In [ ]:
from IPython.display import clear_output

class CustomCallback(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.x = []
        self.loss = []
        self.val_loss = []


    def on_test_batch_begin(self, epoch, logs={}):
        pass

    def on_epoch_end(self, epoch, logs={}):
        self.x.append(epoch)
        self.loss.append(logs["loss"])

        clear_output(wait=True)
        tf.print(logs)
        self.val_loss.append(logs["val_loss"])
        plt.figure(figsize=(10, 8))
        ax1 = plt.subplot2grid((1,1), (0,0), colspan=1, rowspan=1)
        ax1.plot(self.x, self.loss, lw=4, label="Training")
        ax1.plot(self.x, self.val_loss, lw=4, label="Validation")
        ax1.legend(fontsize=16)
        ax1.set_xlabel("Epoche", fontsize=16)
        ax1.set_ylabel("Loss", fontsize=16)

        plt.tight_layout()
        plt.show()


custom_callback = CustomCallback()

In [ ]:
model.fit(x_train, y_train, validation_split=0.1 ,epochs=3, callbacks=[custom_callback], steps_per_epoch=100)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [ ]:
for i in a.take(1):
    print(i)

In [ ]:
a = dataset.map(lambda pic, label: {"digits": pic, "label": label, "thingy": 1})

In [ ]:
for i in a.take(1):
    print(a["label"])

In [ ]:
model.fit(a)

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

In [ ]:
background = Image.open("test_image.png")

In [ ]:
background.convert("RGB")

In [ ]:
x_spacing = 23
y_spacing = 28

In [ ]:
import numpy as np

im = np.zeros((225,225,3), dtype=np.uint8)
# Draw grid
im[:, y_spacing:-1:y_spacing] = [255,0,0]    # red horizontal lines
im[x_spacing:-1:x_spacing, :] = [255,0,0]    # blue vertical lines
new_ima = Image.fromarray(im).convert("RGB")

In [ ]:
new_img = Image.blend(background.convert("RGB"), new_ima, 0.5)

In [ ]:
new_img.show()

In [ ]:
new_img = Image.blend(background.convert("RGB"), new_ima, 0.5)
ImageDraw.Draw(new_img).text(
    (0 * y_spacing + 2, 0 * x_spacing),  # Coordinates
    '1',
    (255, 0,0) # Color
)
new_img.save("saved_test.png")

In [ ]:
loss_wert = 0.8
if loss_wert > 0.5:
    color = (255,0,0)

font = ImageFont.load_default()

In [ ]:
font

In [ ]:
font = ImageFont.truetype("/root/pyscripts/arial.ttf", 12)